In [14]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [15]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [16]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

True

### Karger Books Para

In [17]:
model_name_or_path = 'facebook/bart-large'

In [18]:
data_dir = '"'+bucket_dir+'datasets/xsum"'

output_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_xsum"'

log_dir = bucket_dir + '/logs'

In [19]:
from transformers import AutoConfig
AutoConfig.from_pretrained('facebook/bart-large-xsum')

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "eos_token_ids": [
    2
  ],
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2":

In [20]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_xsum.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_xsum.json"'

In [28]:
!python3 $eval_script \
'facebook/bart-large-xsum' \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--fp16 \
--bs 16 \
--dump-args

100%|███████████████████████████████████████| 709/709 [1:05:24<00:00,  5.54s/it]
{'rouge1': 45.3773, 'rouge2': 22.3115, 'rougeL': 37.1896, 'rougeLsum': 37.1944, 'n_obs': 11333, 'runtime': 3930, 'seconds_per_sample': 0.3468}


In [40]:
AutoConfig.from_pretrained('facebook/bart-large-xsum')

BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "eos_token_ids": [
    2
  ],
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2":

In [48]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained('facebook/bart-large', cache_dir=cache_dir)
model_config.activation_dropout = 0
model_config.attention_dropout = 0
model_config.classif_dropout = 0
model_config.min_length = 11
model_config.max_length = 63
model_config.no_repeat_ngram_size = 3
model_config.num_beams = 6
model_config.task_specific_params['summarization']['length_penalty'] = 1
model_config.task_specific_params['summarization']['min_length'] = 11
model_config.task_specific_params['summarization']['max_length'] = 62
model_config.task_specific_params['summarization']['num_beams'] = 6

model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    'facebook/bart-large'.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

##### Fine tune

In [35]:
!python3 $finetune_script --help

usage: finetune_trainer.py [-h] --model_name_or_path MODEL_NAME_OR_PATH
                           [--config_name CONFIG_NAME]
                           [--tokenizer_name TOKENIZER_NAME]
                           [--cache_dir CACHE_DIR] [--freeze_encoder]
                           [--freeze_embeds] --data_dir DATA_DIR [--task TASK]
                           [--max_source_length MAX_SOURCE_LENGTH]
                           [--max_target_length MAX_TARGET_LENGTH]
                           [--val_max_target_length VAL_MAX_TARGET_LENGTH]
                           [--test_max_target_length TEST_MAX_TARGET_LENGTH]
                           [--n_train N_TRAIN] [--n_val N_VAL]
                           [--n_test N_TEST] [--src_lang SRC_LANG]
                           [--tgt_lang TGT_LANG] [--eval_beams EVAL_BEAMS]
                           [--no_ignore_pad_token_for_loss] --output_dir
                           OUTPUT_DIR [--overwrite_output_dir] [--do_train]
                       

In [56]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--fp16 \
--attention_dropout 0 --dropout 0.1 \
--learning_rate 1.2e-4 \
--sortish_sampler --freeze_embeds \
--task summarization \
--max_source_length 1024 \
--max_target_length 60 \
--val_max_target_length 60 \
--do_train \
--max_steps 5000 \
--logging_steps 100 --save_steps 1000 --save_total_limit 3 \
--per_device_train_batch_size 2 --per_device_eval_batch_size 2 \
--gradient_accumulation_steps 64 \
--do_eval --evaluation_strategy steps --eval_steps 1000 \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

01/15/2021 18:10:55 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
01/15/2021 18:10:55 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/facebook?bart-large_xsum', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=64, eval_accumulation_steps=None, learning_rate=0.00012, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=5000, warmup_steps=0, logging_dir='runs/Jan15_18-10-55_ip-172-31-39-35', logging_first_step=False, logging_steps=100, save_steps=1000, save_total_limit=3, no_cuda=False, seed=42, fp16

wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.13
wandb: Syncing run /home/ubuntu/s3/fine-tuning/facebook?bart-large_xsum
wandb: ⭐️ View project at https://wandb.ai/marcoabrate/huggingface
wandb: 🚀 View run at https://wandb.ai/marcoabrate/huggingface/runs/2h4md3hy
wandb: Run data is saved locally in /home/ubuntu/magma/fine-tuning/wandb/run-20210115_181131-2h4md3hy
wandb: Run `wandb offline` to turn off syncing.

  0%|          | 6/5000 [01:33<20:47:13, 14.98s/it]^C
Traceback (most recent call last):
  File "/home/ubuntu/transformers/examples/seq2seq/finetune_trainer.py", line 379, in <module>
    main()
  File "/home/ubuntu/transformers/examples/seq2seq/finetune_trainer.py", line 316, in main
    model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
  File "/home/ubuntu/transformers/src/transformers/trainer.py", line 799, in train
    tr_loss += self.train

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = huggingface bart large xsum has one more tokenoutput_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--bs 2 \
--length_penalty $config.LENAGTH_PENALTY \
--no_repeat_ngram_size $config.NO_REPEAT_NGRAM_SIZE \
--num_beams $config.NUM_BEAMS \
--dump-args